In [1]:
#Instalando as bibliotecas
#%pip install pandas

## Importar as bibliotecas
import pandas as pd

In [15]:
## Carregar o arquivo csv
df2024 = pd.read_csv(r'c:\Repos\tcc-bruna\fonte.csv')
df2019 = pd.read_excel(r'C:\Repos\tcc-bruna\verificadoativos07-05-2019abr-2019.xlsx')

C:\Users\freda\AppData\Local\Temp\ipykernel_49880\3706224011.py:2: DtypeWarning: Columns (0,1,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df2024 = pd.read_csv(r'c:\Repos\tcc-bruna\fonte.csv')


In [16]:
## Comparar a quantidade de servidores entre os dois dataframes, nas bibliotecas
df2024 = df2024.dropna(subset=['SETOR'])
df2019 = df2019.dropna(subset=['SETOR'])

## Vamos remover todas as observações que não possuem a substring ''BIBLIOTECA'' na coluna 'SETOR'
df_bibliotecas24 = df2024[df2024['SETOR'].str.contains('BIBLIOTECA')]
df_bibliotecas19 = df2019[df2019['SETOR'].str.contains('BIBLIOTECA')]

##Remover as linhas cuja variável 'SETOR' é BIBLIOTECA MUNICIPAL MARIO DE ANDRADE , BIBLIOTECA PUBLICA MUNICIPAL LOUIS BRAILLE, BIBLIOTECA PUBLICA MUNICIPAL SERGIO MILLIET, BIBLIOTECA JOAO CABRAL DE MELO NETO (CEU), BIBLIOTECA JORNALISTA ROBERTO MARINHO (CEU) e BIBLIOTECA RACHEL DE QUEIROZ (CEU)
excluir = [
    'BIBLIOTECA MUNICIPAL MARIO DE ANDRADE',
    'BIBLIOTECA PUBLICA MUNICIPAL LOUIS BRAILLE',
    'BIBLIOTECA PUBLICA MUNICIPAL SERGIO MILLIET',
    'BIBLIOTECA JOAO CABRAL DE MELO NETO',
    'BIBLIOTECA JORNALISTA ROBERTO MARINHO',
    'BIBLIOTECA RACHEL DE QUEIROZ'
]

pattern = '|'.join(excluir)

df_bibliotecas24 = df_bibliotecas24[~df_bibliotecas24['SETOR'].str.contains(pattern)]

df_bibliotecas19 = df_bibliotecas19[~df_bibliotecas19['SETOR'].str.contains(pattern)]




In [30]:
def comparar_data_exerc(df2024, df2019):
    ## Converter a coluna 'DATA_EXERCICIO' para datetime
    df2024['DATA_INICIO_EXERC'] = pd.to_datetime(df2024['DATA_INICIO_EXERC'], errors='coerce')
    df2019['DATA_INICIO_EXERC'] = pd.to_datetime(df2019['DATA_INICIO_EXERC'], errors='coerce')    
    ## Criar uma nova tabela com o nome do servidor e a data de exercício do df2024 e df2019. Caso o servidor não esteja presente no dataframe de 2024, a data de exercício será nula.
    df_data_exerc = pd.merge(df2024[['NOME', 'DATA_INICIO_EXERC']], df2019[['NOME', 'DATA_INICIO_EXERC']], on='NOME', how='left', suffixes=('_2024', '_2019'))

    return df_data_exerc

df_data_exerc_bibliotecas = comparar_data_exerc(df_bibliotecas24, df_bibliotecas19)

def identificando_consistencia_de_data_inic_exer(df_data_exerc):
    ## Identificar as mudanças
    df_data_exerc['CONSISTENTE'] = df_data_exerc['DATA_INICIO_EXERC_2024'] == df_data_exerc['DATA_INICIO_EXERC_2019']
    ## Filtrar as mudanças
    df_mudancas = df_data_exerc[df_data_exerc['CONSISTENTE'] == True]
    ## Selecionaremos todos os regis

    return df_data_exerc

df_incons = identificando_consistencia_de_data_inic_exer(df_data_exerc_bibliotecas)

def resolver_inconsistencias(df_incons):
    ## Analizaremos a tabela de inconsistências criando uma nova coluna "DATA_INICIO_EXERC" que será preenchida com o valor da coluna "DATA_INICIO_EXERC_2019" caso a coluna "CONSISTENTE" seja False e o valor da coluna "DATA_INICIO_EXERC_2024" não seja nulo. Nos casos em que a coluna "CONSISTENTE" seja False, e o valor da coluna "DATA_INICIO_EXERC_2019" seja nulo, a coluna "DATA_INICIO_EXERC" será preenchida com o valor da coluna "DATA_INICIO_EXERC_2024".

    df_incons['DATA_INICIO_EXERC'] = df_incons['DATA_INICIO_EXERC_2019']
    df_incons.loc[df_incons['DATA_INICIO_EXERC'].isnull(), 'DATA_INICIO_EXERC'] = df_incons['DATA_INICIO_EXERC_2024']

    ## Removeremos as colunas "DATA_INICIO_EXERC_2024", "DATA_INICIO_EXERC_2019" e "CONSISTENTE"
    df_incons = df_incons.drop(columns=['DATA_INICIO_EXERC_2024', 'DATA_INICIO_EXERC_2019', 'CONSISTENTE'])

    return df_incons
    
       

df_resolvido = resolver_inconsistencias(df_incons)

## Criaremos uma função que vai substituir os dados da coluna data de início de exercício do df2024 pelos dados resolvidos
def introduzir_dados_resolvidos(df_resolvido, df_bibliotecas24):
    ##Para cada observação da tabela de dados resolvidos, vamos substituir o valor da coluna "DATA_INICIO_EXERC" na tabela de df_bibliotecas24
    for i in df_resolvido.index:
        nome = df_resolvido.loc[i, 'NOME']
        df_bibliotecas24.loc[df_bibliotecas24['NOME'] == nome, 'DATA_INICIO_EXERC'] = df_resolvido.loc[i, 'DATA_INICIO_EXERC']
    ## Retornar a tabela de df_bibliotecas24
    
    return df_bibliotecas24

df_bibliotecas24_resolvido = introduzir_dados_resolvidos(df_resolvido, df_bibliotecas24)

## Criaremos uma função que vai identificar os servidores e servidoras que poderão se aposentar em 2024, para isso consideraremos as seguintes regras de transição (categoria pedágio): Para amnbos sexos, ter mais de 20 anos de serviço público. Para homens, ter mais de 60 anos de idade. Para mulheres, ter mais de 57 anos de idade.

def identificar_aposentadoria(df_bibliotecas24_resolvido):
    
    ## Criar uma nova coluna "IDADE_EM_2024" que será preenchida com a diferença entre o ano de 2024 e o ano de nascimento do servidor
    df_bibliotecas24_resolvido['IDADE_EM_2024'] = 2024 - df_bibliotecas24_resolvido['ANO_NASCIMENTO'].astype(int)
    ## Criar uma nova coluna "TEMPO_DE_SERVICO_EM_2024" que será preenchida com a diferença entre o ano de 2024 e o ano de início do exercício do servidor
    df_bibliotecas24_resolvido['TEMPO_DE_SERVICO_EM_2024'] = 2024 - df_bibliotecas24_resolvido['DATA_INICIO_EXERC'].dt.year
    ## Criar uma nova coluna "APTA_A_APOSENTAR" que será preenchida com True caso o servidor tenha mais de 20 anos de serviço público e mais de 60 anos de idade (homens) ou mais de 57 anos de idade (mulheres)
    df_bibliotecas24_resolvido['APTA_A_APOSENTAR'] = (df_bibliotecas24_resolvido['TEMPO_DE_SERVICO_EM_2024'] >= 20) & ((df_bibliotecas24_resolvido['IDADE_EM_2024'] >= 60) | ((df_bibliotecas24_resolvido['IDADE_EM_2024'] >= 57) & (df_bibliotecas24_resolvido['SEXO'] == 'F')))
    ## Filtrar as observações que são aptas a se aposentar
    df_aposentadoria = df_bibliotecas24_resolvido[df_bibliotecas24_resolvido['APTA_A_APOSENTAR'] == True]
    ## Retornar a tabela de df_aposentadoria
    return df_aposentadoria

df_aposentadoria = identificar_aposentadoria(df_bibliotecas24_resolvido)

In [16]:
##Mostrar as informações dos setores do ano de 2024 e 2019
def contar_servidores_por_setor(df, ano):
    setores_contagem = df['SETOR'].value_counts().reset_index()
    setores_contagem.columns = ['SETOR', 'QUANTIDADE']
    setores_contagem = setores_contagem.sort_values(by='QUANTIDADE', ascending=False)
    print(f'Quantidade de servidores por setor em {ano}:\n{setores_contagem}')
    return setores_contagem

setores_contagem_24 = contar_servidores_por_setor(df_bibliotecas24, 2024)
setores_contagem_19 = contar_servidores_por_setor(df_bibliotecas19, 2019)

##TODO: Verificar se a data de início no df 2024 é o mesmo do df 2019 para todos os servidores, se for diferente considerar a data de início do df 2019








Quantidade de servidores por setor em 2024:
                                                SETOR  QUANTIDADE
0                           SUPERVISAO DE BIBLIOTECAS          36
1     COORDENACAO DO SISTEMA MUNICIPAL DE BIBLIOTECAS          14
2        BIBLIOTECA PUBLICA MUNICIPAL MONTEIRO LOBATO          12
3        BIBLIOTECA PUBLICA MUNICIPAL AFONSO SCHIMIDT          11
4    BIBLIOTECA PUBLICA MUNICIPAL RAIMUNDO DE MENEZES           9
5     BIBLIOTECA PUBLICA MUNICIPAL VINICIUS DE MORAES           8
6          BIBLIOTECA PUBLICA MUNICIPAL RICARDO RAMOS           8
7   BIBLIOTECA PUBLICA MUNICIPAL PE. JOSE DE ANCHIETA           8
8   BIBLIOTECA PUBLICA MUNICIPAL JOVINA ROCHA ALVARES           8
9     BIBLIOTECA PUBLICA MUNICIPAL ALVARES DE AZEVEDO           8
10  BIBLIOTECA PUBLICA MUNICIPAL HANS CHRISTIAN ANDER           7
11         BIBLIOTECA PUBLICA MUNICIPAL CORA CORALINA           7
17            BIBLIOTECA PUBLICA MUNICIPAL ANNE FRANK           6
20  BIBLIOTECA PUBLICA MUNICIPAL

In [14]:
##  Criaremos uma tabela comparativa entre os dois anos, mostrando a quantidade de servidores em cada setor em 2019 e 2024
comparativo = pd.merge(setores_contagem_19, setores_contagem_24, on='SETOR', how='outer')
comparativo.columns = ['SETOR', '2019', '2024']
comparativo = comparativo.fillna(0)
comparativo['2019'] = comparativo['2019'].astype(int)
comparativo['2024'] = comparativo['2024'].astype(int)
comparativo = comparativo.sort_values(by='2024', ascending=False)
print(f'Comparativo entre 2019 e 2024:\n{comparativo}')

## Queremos uma tabela com o total de servidores em 2019 e 2024
total_2019 = setores_contagem_19['QUANTIDADE'].sum()
total_2024 = setores_contagem_24['QUANTIDADE'].sum()
print(f'Total de servidores em 2019: {total_2019}')
print(f'Total de servidores em 2024: {total_2024}')


Total de servidores em 2019: 363
Total de servidores em 2024: 339


In [17]:
## Vamos visualizar a distribuição espacial desses servidores
%pip install geopandas
%pip install matplotlib

import matplotlib.pyplot as plt
import geopandas as gpd

## Carregar o arquivo shapefile
mapa_setores = gpd.read_file(r'C:\Repos\tcc-bruna\DEINFO_DISTRITO.shp')

endereco_setores = []




Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/323.6 kB ? eta -:--:--
   --------------------------------------- 323.6/323.6 kB 10.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.7/15.9 MB 55.5 MB/s eta 0:00:01
   ----------- ---------------------------- 4.7/15.9 MB 59.9 MB/s eta 0:00:01
   ------------------ --------------------- 7.3/15.9 MB 58.1 MB/s eta 0:00:01
   -------------------------- ------------- 10.4/15.9 MB 59.8 MB/s eta 0:00:01
   -------------------------------- ------- 13.1/15.9 MB 59.5 MB/s eta 0:00:01
   ---------------------------------------  15.9/15.9 MB 59.5 MB/s eta 0:00:01
   ---------------------------------------- 15.9/15.9 MB 50.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ------------- -------------------------- 2.1/6.1 MB 66.1 MB/s eta 0:00:01
   --

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
